# Step 1: Setup

- Download the `sample_ovules.h5` from the test data folder.
- Run the following cell to create a mock dataset for training our custom plantseg model.

In [ ]:
from plantseg.io.voxelsize import VoxelSize
from plantseg.io import smart_load, create_h5
from pathlib import Path

path = Path("sample_ovules.h5")

image = smart_load(path, key="raw")
label = smart_load(path, key="label")

train_size = 2
val_size = 2
test_size = 2

for i, (im2d, lab2d) in enumerate(zip(image, label)):
    
    if i < val_size:
        group = "val"
    elif i < val_size + test_size:
        group = "test"
    elif i < val_size + test_size + train_size:
        group = "train"    
    else:
        break
    
    path = Path(f"./training/{group}/ovule_2d_{i:03d}.h5")
    path.parent.mkdir(parents=True, exist_ok=True)
    create_h5(path, stack=im2d[None, ...], key="raw", voxel_size=VoxelSize(voxels_size=(1, 1, 1)))
    create_h5(path, stack=lab2d[None, ...], key="label", voxel_size=VoxelSize(voxels_size=(1, 1, 1)))

INFO: P [MainThread] 2025-03-11 22:45:53,608 plantseg - Logger configured at initialisation. PlantSeg logger name: plantseg


NameError: name 'smart_load' is not defined

# Step 2: Download the a template config file, and modify it to fit the mock dataset
- Download the template config file from [here](https://zenodo.org/records/7805067) and save it as `config.yaml`.
- Modify the `config.yaml` file to fit the mock dataset.
    In particular, you need to modify the following fields:
```yaml
  validate_after_iters: Some small number (e.g., 3)
  log_after_iters: Some small number (e.g., 1)

  train:
    # paths to the training datasets
    file_paths:
      - "Path to the mock dataset train dir"
  ...
  val:
    # paths to the validation datasets
    file_paths:
    - "Path to the mock dataset val dir"
```

# Step 3: Run the training using pytorch3dunet

In the terminal, run the following command to train the model:
```bash
train3dunet --config PATH_TO_CONFIG_FILE
```

# Step 4: Follow the training progress using tensorboard

In the terminal, run the following command to start tensorboard:
```bash
tensorboard --logdir PATH_TO_LOG_DIR (./logs by default)
```

Then, open a browser and go to `http://localhost:6006/` to see the training progress.

# Step 5: Play with the config file parameters

- You can play with the config file parameters to see how they affect the training process.
- For example, you can change the learning rate, batch size, number of epochs, etc.

# Step 6: Fine-tune from an existing model

- Download the pre-trained model from TODO
- Modify the `config.yaml` file to use the pre-trained model:
```yaml
trainer:
  pre_trained: "Path to the pre-trained model"
```

# Step 7: Add the model to the plantseg Model Zoo using the gui